In [23]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from tensorflow import keras

#import DataGenerator as DG
from DataGenerator import DataGenerator

import sklearn.metrics as metrics

import numpy as np
import random
import math
import os

import matplotlib.pyplot as plt

import h5py

###FIX NUMPY LOAD FOR DICTIONARIES
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
depth = 8

In [12]:
model_files=["model-2019-07-24-16:16:35.h5","model-2019-07-24-16:47:28.h5","model-2019-07-25-12:57:07.h5"]
#,"model-2019-07-24-19:30:14.h5"
filepath = "images_raw_doric_round1.h5"

scratch_dir = "/n/scratch2/ap487/"
if not os.path.exists((scratch_dir+filepath)):
    copy(filepath,scratch_dir)
    print("File Copied to scratch directory")
filepath=scratch_dir+filepath
print("Using path: ",filepath)

Using path:  /n/scratch2/ap487/images_raw_doric_round1.h5


In [13]:
def accuracy(predictions, answers, top_n=5):
    accs = np.zeros(len(answers))
    predicts = np.zeros(len(test_labels))
    for i in range(0,len(answers)):
        j = len(predictions[i])-1
        for k in range(0,top_n):
            if predictions[i][j-k]==answers[i]:
                accs[i]=1
                predicts[i]=predictions[i][j-k]
    return np.mean(accs), predicts

In [35]:
with h5py.File(filepath,'r') as f:
    test_data=np.zeros((1000,depth,80,80,1))
    #test_data = test_data[...,None]
    test_labels=np.zeros(1000)
    i=0
    #Starting index for batch samples
    index = 0
    data_gen = DataGenerator(filepath,data_amount=len(f["/labels"]),batch_size=1000,frames_per_sample=depth)
    while i < 1:
        #Loop forward to add to the frame sequence
        #cur_amount = 0
        #while cur_amount < depth:
        #    label = f["/labels"][index]
        #    if label >= 0:
        #        test_data[i][cur_amount]=f["/frames/raw"][index]
        #        test_labels[i]=f["/labels"][index]
        #        index+=1
        #        cur_amount+=1
        #    else:
        #        index+=1
        dat,lbl = data_gen.__getitem__(i)
        print(test_data[i*1000:(i+1)*1000].shape)
        test_data[i*1000:(i+1)*1000]=dat
        test_labels[i*1000:(i+1)*1000]=lbl
        i+=1

(1000, 8, 80, 80, 1)


In [40]:
models = []

In [42]:
for i in range(0,len(model_files)):
    p = keras.models.load_model(model_files[i]).predict(test_data)
    for j in range(0,len(p)):
        p[j]=np.argsort(p[j])
    conf_mat=metrics.confusion_matrix(test_labels,[x[(len(x)-1)] for x in p])
    conf_mat=conf_mat / conf_mat.astype(np.float).sum(axis=1,keepdims=True)
    plt.rcParams["figure.figsize"] = (15,15)
    plt.imshow(conf_mat)
    plt.xlabel("Prediction")
    plt.ylabel("Actual")
    plt.show()

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[32,8,8,40,40] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node max_pooling3d_13/MaxPool3D}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[32,8,8,40,40] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node max_pooling3d_13/MaxPool3D}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[activation_2_11/Softmax/_2603]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successful operations.
0 derived errors i

In [39]:
!nvidia-smi

Thu Jul 25 13:15:59 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.48                 Driver Version: 410.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:04:00.0 Off |                    0 |
| N/A   67C    P0   147W / 149W |  10943MiB / 11441MiB |     97%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:05:00.0 Off |                    0 |
| N/A   53C    P0   137W / 149W |  10941MiB / 11441MiB |     78%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T